<a href="https://colab.research.google.com/github/PiyushTewari2/squeezenet-pytorch/blob/main/batch_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/squeezenet_fix_batch.py .
import squeezenet_fix_batch
!unzip '/content/drive/MyDrive/squeeze-net/Data.zip'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/squeeze-net/Data.zip
replace Data/Test_Data/Fire/F_0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Data/Test_Data/Fire/F_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [5]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


In [13]:
data_transforms = {
    'Train_Data': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Test_Data': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [14]:
# Define the data directory
data_dir = '/content/Data'

# Create data loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['Train_Data', 'Test_Data']}
#image_datasets
#creating datasets
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['Train_Data', 'Test_Data']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['Train_Data', 'Test_Data']}
print(dataset_sizes)

class_names = image_datasets['Train_Data'].classes
class_names

{'Train_Data': 4516, 'Test_Data': 50}


['Fire', 'Non_Fire']

In [15]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    # plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
# inputs, classes = next(iter(dataloaders[TRAIN]))
# show_databatch(inputs, classes)

def visualize_model(squeezenet, num_images=6):
    was_training = squeezenet.training

    # Set model for evaluation
    squeezenet.train(False)
    squeezenet.eval()

    images_so_far = 0

    for i, data in enumerate(dataloaders['Test_Data']):
        inputs, labels = data
        size = inputs.size()[0]

        with torch.no_grad():
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            else:
                inputs, labels = inputs, labels

                outputs = squeezenet(inputs)

                _, preds = torch.max(outputs.data, 1)
                predicted_labels = [preds[j] for j in range(inputs.size()[0])]

                print("Ground truth:")
                show_databatch(inputs.data.cpu(), labels.data.cpu())
                print("Prediction:")
                show_databatch(inputs.data.cpu(), predicted_labels)

        images_so_far += size
        if images_so_far >= num_images:
            break

    squeezenet.train(mode=was_training) # Revert model back to original training state

def eval_model(squeezenet, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0

    test_batches = len(dataloaders['Test_Data'])
    print("Evaluating model")
    print('-' * 10)

    squeezenet.train(False)
    squeezenet.eval()

    with torch.no_grad():
        for i, data in enumerate(dataloaders['Test_Data']):
            if i % 10 == 0:
                print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)


            inputs, labels = data
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            else:
                inputs, labels = inputs, labels

            outputs = squeezenet(inputs)

            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            loss_test += loss.data
            acc_test += torch.sum(preds == labels.data).item()

            # del inputs, labels, outputs, preds
            # torch.cuda.empty_cache()

    avg_loss = loss_test / dataset_sizes['Test_Data']
    avg_acc = acc_test / dataset_sizes['Test_Data']

    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)

In [16]:
## MODEL CREATION

# Load the pretrained model from pytorch
#squeezenet1_1 = models.squeezenet1_1()
squeezenet1_1 = squeezenet_fix_batch.squeezenet1_1()

#squeezenet1_1.load_state_dict(torch.load("/content/drive/MyDrive/squeezenet_v2.pt"))
print(squeezenet1_1.classifier[1].out_channels) # 1000

# Freeze training for all layers
for param in squeezenet1_1.features.parameters():
    param.require_grad = False


# Newly created modules have require_grad=True by default
num_features = squeezenet1_1.classifier[1].in_channels
features = list(squeezenet1_1.classifier.children())[:-3] # Remove last 3 layers

features.extend([nn.AdaptiveAvgPool2d(output_size=(1,1))])
features.extend([nn.Flatten()]) # Add
features.extend([nn.Linear(num_features, len(class_names))]) # Add
features.extend([nn.Softmax(dim=1)]) # Add
squeezenet1_1.classifier = nn.Sequential(*features) # Replace the model classifier

# If you want to train the model for more than 2 epochs, set this to True after the first run
resume_training = False

if use_gpu:
    squeezenet1_1.cuda() #.cuda() will move everything to the GPU side

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(squeezenet1_1.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

#print("Test before training")
#eval_model(squeezenet1_1, criterion)

1000


In [17]:
print(squeezenet1_1)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (4): Fire(
      (squeeze): Sequential(
        (0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Sequential(
        (0): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [18]:
def train_model(squeezenet, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(squeezenet.state_dict())
    best_acc = 0.0

    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0

    train_batches = len(dataloaders['Train_Data'])
    val_batches = len(dataloaders['Test_Data'])

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)

        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0

        squeezenet.train(True)

        for i, data in enumerate(dataloaders['Train_Data']):
            if i % 10 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)

            # Use half training dataset
            if i >= train_batches / 2:
                break

            inputs, labels = data

            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            else:
                inputs, labels = inputs, labels

            optimizer.zero_grad()

            outputs = squeezenet(inputs)

            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            loss_train += loss.data
            acc_train += torch.sum(preds == labels.data).item()

            # del inputs, labels, outputs, preds
            # torch.cuda.empty_cache()

        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train * 2 / dataset_sizes['Train_Data']
        avg_acc = acc_train * 2 / dataset_sizes['Train_Data']

        squeezenet.train(False)
        squeezenet.eval()

        for i, data in enumerate(dataloaders['Test_Data']):
            if i % 10 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)

            inputs, labels = data

            with torch.no_grad():
                if use_gpu:
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = inputs, labels

                optimizer.zero_grad()

                outputs = squeezenet(inputs)

                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                loss_val += loss.data
                acc_val += torch.sum(preds == labels.data).item()

                # del inputs, labels, outputs, preds
                # torch.cuda.empty_cache()

        avg_loss_val = loss_val / dataset_sizes['Test_Data']
        avg_acc_val = acc_val / dataset_sizes['Test_Data']

        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()

        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(squeezenet.state_dict())

    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))

    squeezenet.load_state_dict(best_model_wts)
    return squeezenet

squeezenet = train_model(squeezenet1_1, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)
torch.save(squeezenet1_1.state_dict(), 'drive/MyDrive/squeezenet_batchnorm_version_standard_training.pt')

Epoch 0/10
----------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training batch 560/564.5
Validation batch 10/13
Epoch 0 result: 
Avg loss (train): 0.1392
Avg acc (train): 0.7467
Avg loss (val): 0.1340
Avg acc (val): 0.8200
----------

Epoch 1/10
----------
Training batch 560/564.5
Validation batch 10/13
Epoch 1 result: 
Avg loss (train): 0.1199
Avg acc (train): 0.8313
Avg loss (val): 0.1234
Avg acc (val): 0.8400
----------

Epoch 2/10
----------
Training batch 560/564.5
Validation batch 10/13
Epoch 2 result: 
Avg loss (train): 0.1145
Avg acc (train): 0.8565
Avg loss (val): 0.1410
Avg acc (val): 0.7600
----------

Epoch 3/10
----------
Training batch 560/564.5
Validation batch 10/13
Epoch 3 result: 
Avg loss (train): 0.1121
Avg acc (train): 0.8667
Avg loss (val): 0.1297
Avg acc (val): 0.8400
----------

Epoch 4/10
----------
Training batch 560/564.5
Validation batch 10/13
Epoch 4 result: 
Avg loss (train): 0.1115
Avg acc (train): 0.8676
Avg loss (val): 0.1417
Avg acc (val): 0.7600
----------

Epoch 5/10
----------
Training batch 560/564.5
Validation